Use Shift+Enter to execute each cell

Try to make our code python-3 friendlier for future implementation and grab the import function

In [1]:
from __future__ import (division,
                        print_function,
                        #unicode_literals,
                       )
from importlib import import_module

Import useful functions we may want

In [2]:
import os, sys, time

We need ROOT's functionality...

In [3]:
import ROOT

Welcome to JupyROOT 6.10/09


Import the PostProcessor, the framework for running on NanoAOD

In [4]:
from PhysicsTools.NanoAODTools.postprocessing.framework.postprocessor import PostProcessor

We want the Module Class, and Collection/Object helper methods

In [5]:
from PhysicsTools.NanoAODTools.postprocessing.framework.datamodel import Collection, Object
from PhysicsTools.NanoAODTools.postprocessing.framework.eventloop import Module

Here we define our class module, which we'll load/configure near the bottom, then finally run. 
This inherits from the base class "Module."
Where we do 'def beginJob' or 'def analyze,' we override the base Module's definition with our own.
We can name our class almost anything, so long as it matches the name that we pass to the PostProcessor(p=PostProcessor(...) definition near the end), which will create an instances of this class to be used in the analysis chain

In [6]:
class OurExampleModule(Module):
    def __init__(self):
        self.writeHistFile=True #Necessary for an output file to be created? 
        self.counter = 0 #Define this global variable to count events
        self.EventLimit = -1
        self.isData = False #For running on data versus MonteCarlo
    def beginJob(self,histFile=None,histDirName=None):
        #beginJob is typically where histograms should be initialized
        #So we call the default Module's beginJob, passing it the histFile and histDirName first passed to the PostProcessor
        Module.beginJob(self,histFile,histDirName)
        #Here, create a 1-D histogram of type Double(TH1D)
        #with histogram_name h_jets, and someTitle(title)/nJets(x-axis)/Events(y-axis), 20 bins, with domain 0 to 20
        self.h_jets = ROOT.TH1D('h_jets', 'someTitle;nJets;Events',   20, 0, 20)
        #The histogram has to be 'booked' with the service that will write everything to the output file
        self.addObject(self.h_jets)
        #Repeat for other histograms
        self.h_fatjets = ROOT.TH1D('h_fatjets', ';nFatJets;Events', 8, 0, 8)
        self.addObject(self.h_fatjets)
        self.h_subjets = ROOT.TH1D('h_subjets', ';nSubJets;Events', 16, 0, 16)
        self.addObject(self.h_subjets)
        
    #We can override the other processes with "pass,"
    #If we do this for endFile, histograms won't be written    
    #def endJob(self):
    #    pass

    #def beginFile(self, inputFile, outputFile):
    #    pass

    #def beginFile(self):
    #    pass

    #def endFile(self, inputFile, outputFile):
    #    pass

    #def endFile(self):
    #    pass
    
    def analyze(self, event):
        #DOC string for our analyze method, denoted by triple-quotes
        """process event, return True (go to next module) or False (fail, go to next event)"""
        
        self.h_jets.Fill(event.nJet)
        self.h_fatjets.Fill(event.nFatJet)
        self.h_subjets.Fill(event.nSubJet)
        
        modulator = 1000 # how often to print an event when we do self.counter % modulator
        self.counter += 1
        
        #Below we 'halt' execution for events past the first N (20 when written) by returning False now 
        if self.counter > self.EventLimit > -1:
            return False
        
        ###########################################
        ###### Basic Attributes of the Event ######
        ###########################################
        #Use basic python getattr() method to grab this info, no need for Object or Collection here
        run = getattr(event, "run")
        lumi = getattr(event, "luminosityBlock")
        evt = getattr(event, "event")
        #if self.counter % modulator == 0:
        print("\n\nRun: {0:>8d} \tLuminosityBlock: {1:>8d} \tEvent: {2:>8d}".format(run,lumi,evt)) 
    
        ###########################################
        ###### Event Collections and Objects ######
        ###########################################
        #Collections are for variable-length objects, easily identified by a nVARIABLE object in the NanoAOD file ("nJet")
        #Objects are for 1-deep variables, like HLT triggers, where there are many of them, but there is only one boolean value
        #for each HLT_SomeSpecificTrigger for each event. These are more than just wrappers, providing convenient methods
        #This will 'work' for anything that has some common name + '_' (like "SV_x" and "SV_y" and "SV_z")
        #Objects:
        met = Object(event, "MET")
        PV = Object(event, "PV")
        HLT = Object(event, "HLT") 
        Filters = Object(event, "Flag")
        #Collections:
        electrons = Collection(event, "Electron")
        photons = Collection(event, "Photon")
        muons = Collection(event, "Muon")
        #taus = Collection(event, "Tau") 
        jets = Collection(event, "Jet")
        fatjets = Collection(event, "FatJet")
        subjets = Collection(event, "SubJet")
        SV = Collection(event, "SV") 
        if not self.isData:
            gens = Collection(event, "GenPart")
            genJets = Collection(event, "GenJet")
            genFatjets = Collection(event, "GenJetAK8")
            genSubjets = Collection(event, "SubGenJetAK8")
    
        ################################
        ###### High Level Trigger ######
        ################################
        passTrig=["IsoMu20", "IsoMu24"] #Create a list of valid triggers to check, dropping "HLT_"
        if self.counter % modulator == 0:
            for trig in passTrig:
                print("\tHLT_" + str(trig) + " Trigger: " + str(getattr(HLT, trig)) )


        ###############################
        ######### MET Filters #########
        ###############################
        if self.isData:
            #All the filters commonly used
            passFilter=["HBHENoiseFilter", "HBHENoiseIsoFilter", "EcalDeadCellTriggerPrimitiveFilter",
                    "globalSuperTightHalo2016Filter", "goodVertices", "METFilters", "noBadMuons"] 
            #if self.counter % modulator == 0:
            for fltr in passFilter:
                print("\t\tFlag_" + str(fltr) + " Filter: " + str(getattr(Filters, fltr)))
                

        ###############################
        ###### Time To Do Stuff! ######
        ###############################
        #Create a TLorentzVector to sum the four-momentum in the event (but don't doublecount by adding Fat/SubJets!)
        eventSum = ROOT.TLorentzVector()
        #Below is a formatted output of the Primary Vertex's coordinates, number of Degrees of Freedom, Chi^2 value
        print("PV  X: {0: >5.3f} Y: {1: >5.3f} Z: {2:5.3f} nDoF: {3: >5.3f} Chi^2: {4: >5.3f}".format(
            PV.x,PV.y, PV.z, PV.ndof, PV.chi2)) #getattr(PV, "chi2") also works
        
        #Now we wish to count and print info about the secondary vertices (heavy flavour hadrons!) in the event, including decay length
        print("==============================================================================")
        print("|| Secondary Vertices\tNumber: {0: >3d} \t\t\t\t\t    ||".format(len(SV)))
        if len(SV) > 0:
            print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{4:>5s}\t{5:>5s}\t{6:>5s}\t\t    ||".format(
                "Pt", "Eta", "Phi", "Chi2", "nDoF", "Mass", "dLen"))
            for vert in SV:
                print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5.3f}\t{5: >5.3f}\t{6: >5.3f}\t\t    ||".format(
                    getattr(vert,"pt"),getattr(vert,"eta"),getattr(vert,"phi"),
                    getattr(vert,"chi2"),getattr(vert,"ndof"),getattr(vert,"mass"),getattr(vert,"dlen")))
                
        #Now the Muons, where JetID is the array index of any jet that matches in eta-phi space (to be used for cross-cleaning)
        print("==============================================================================")
        print("|| Muons\t\tNumber: {0: >3d}\t\t\t\t\t    ||".format(len(muons)))
        if len(muons) > 0:
            print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{4:>5s}\t{5:>5s}\t{6:>5s}\t{7:>5s}  ||".format(
                    "Pt", "Eta", "Phi", "IP3d", "dXY", "dZ", "JetID", "PFRelIso04"))
            for lep in muons:
                eventSum += lep.p4()
                print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5.3f}\t{5: >5.3f}\t{6: >5d}\t{7: >5.3f}\t    ||".format(
                        getattr(lep,"pt"),getattr(lep,"eta"),getattr(lep,"phi"),getattr(lep,"ip3d"),
                        getattr(lep,"dxy"),getattr(lep,"dz"),getattr(lep,"jetIdx"),getattr(lep,"pfRelIso04_all")))
        print("==============================================================================")
        print("|| Electrons\t\tNumber: {0: >3d}\t\t\t\t\t    ||".format(len(electrons)))
        if len(electrons) > 0:
            print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{4:>5s}\t{5:>5s}\t{6:>5s}\t{7:>5s}  ||".format(
                    "Pt", "Eta", "Phi", "IP3d", "dXY", "dZ", "JetID", "PFRelIso03"))
            for lep in electrons:
                eventSum += lep.p4()
                print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5.3f}\t{5: >5.3f}\t{6: >5d}\t{7: >5.3f}\t    ||".format(
                        getattr(lep,"pt"),getattr(lep,"eta"),getattr(lep,"phi"),getattr(lep,"ip3d"),
                        getattr(lep,"dxy"),getattr(lep,"dz"),getattr(lep,"jetIdx"),getattr(lep,"pfRelIso03_all")))
        #Now the photons. Note whether it passes electronVeto, the electron it may be ID-linked with, etc.
        print("==============================================================================")
        print("|| Photons\t\tNumber: {0: >3d}\t\t\t\t\t    ||".format(len(photons)))
        if len(photons) > 0:
            print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{5:>5s}\t{6:>5s}\t{7:>5s}\t{4:>5s}  ||".format(
                    "Pt", "Eta", "Phi", "mvaID", "PFRelIso03", "e ID", "JetID", "eVeto"))
            for gamma in photons:
                #eventSum += gamma.p4()
                print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5d}\t{5: >5d}\t{6: >5d}\t{7: >5.3f}\t    ||".format(
                        getattr(gamma,"pt"),getattr(gamma,"eta"),getattr(gamma,"phi"),getattr(gamma,"mvaID_WP80"),
                        getattr(gamma,"electronIdx"),getattr(gamma,"jetIdx"),getattr(gamma,"electronVeto"),
                        getattr(gamma,"pfRelIso03_all")))
        #Below we'll print info like the CombinedSecondaryVertex Version 2 b-tagging output, the charged electromagnetic and hadron energy fractions...
        print("==============================================================================")
        #for j in filter(self.jetSel,jets):
        print("|| AK4 Jets\t\tNumber: {0: >3d}\t\t\t\t\t    ||".format(len(jets)))
        print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{4:>5s}\t{5:>5s}\t{6:>5s}\t{7:>5s}       ||".format(
                "Pt", "Eta", "Phi", "CSVv2", "CMVA", "JetID", "ChEmEF", "ChHEF"))
        for jet in jets:
            eventSum += jet.p4()
            print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5.3f}\t{5: >5d}\t{6: >5.3f}\t{7: >5.3f}\t    ||".format(
                    getattr(jet,"pt"),getattr(jet,"eta"),getattr(jet,"phi"),getattr(jet,"btagCSVV2"),
                    getattr(jet,"btagCMVA"),getattr(jet,"jetId"),getattr(jet,"chEmEF"),getattr(jet,"chHEF")))
        print("==============================================================================")
        print("|| AK8 Jets\t\tNumber: {0: >3d}\t\t\t\t\t    ||".format(len(fatjets)))
        if len(fatjets) > 0:
            print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{4:>5s}\t{5:>5s}\t{6:>5s}\t{7:>5s}       ||".format(
                    "Pt", "Eta", "Phi", "CSVv2", "Mass", "MSDrp", "sJID1", "sJID2"))
            for fjet in fatjets:
                #Don't sum these in the event, as they're just reclustering of the same energy deposits used to construct "jets"
                print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5.3f}\t{5: >5.3f}\t{6: >5d}\t{7: >5d}\t    ||".format(
                        getattr(fjet,"pt"),getattr(fjet,"eta"),getattr(fjet,"phi"),getattr(fjet,"btagCSVV2"),
                        getattr(fjet,"mass"),getattr(fjet,"msoftdrop"),getattr(fjet,"subJetIdx1"),
                        getattr(fjet,"subJetIdx2")))
        print("==============================================================================")
        print("|| AK8 SubJets\t\tNumber: {0: >3d}\t\t\t\t\t    ||".format(len(subjets)))
        if len(subjets) > 0:
            print("||\t{0:>5s}\t{1:>5s}\t{2:>5s}\t{3:>5s}\t{4:>5s}\t\t\t\t    ||".format(
                    "Pt", "Eta", "Phi", "CSVv2","Mass"))
            for sjet in subjets:
                #Ditto here, no sum, since these should correspond to "jets" in the AK4 collection
                print("||\t{0: >5.3f}\t{1: >5.3f}\t{2: >5.3f}\t{3: >5.3f}\t{4: >5.3f}\t\t\t\t    ||".format(
                        getattr(sjet,"pt"),getattr(sjet,"eta"),getattr(sjet,"phi"),getattr(sjet,"btagCSVV2"),
                        getattr(sjet,"mass")))
        print("==============================================================================")
        if not self.isData:
            print("|| Gen particles\tNumber: {0: >3d}\t\t\t\t\t    ||".format(len(gens)))
            print("|| AK4 genjets\t\tNumber: {0: >3d}\t\t\t\t\t    ||".format(len(genJets)))
            print("|| AK8 genjets\t\tNumber: {0: >3d}\t\t\t\t\t    ||".format(len(genFatjets)))
            print("|| AK8 genSubjets\tNumber: {0: >3d}\t\t\t\t\t    ||".format(len(genSubjets)))
            print("==============================================================================")
        print("Event Mass: {:<10.4f}\n".format(eventSum.M()))
        
        ###########################################
        ###### Return True to pass the event ######
        ###########################################
        return True


We can do a preselection on any element in the file, e.g. requiring at least one detectable lepton shows up, like so...

In [7]:
preselection="nMuon > 0 || nElectron > 0"

#For when we want to loop over every event, and let the analyzer choose to pass/fail the event:
#preselection=None 

Here, define the location where we'll output any root files, "." meaning the current directory from which the code is run.

In [8]:
outputDir = "."
#outputDir = "../path/of/choice"

Create our list of files to be processed, first by creating a prefix for our files list, because we copy those directly from Data Aggragation Service, and it does not include a server reference for the files. Postfix should be paired with the corresponding inputList, which will be added to any root files output (either histogram file or full NanoAOD file of events passing full analyze chain ("return True")

In [9]:
filePrefix = "root://cms-xrd-global.cern.ch/"
files=[]
#Open the text list of files as read-only ("r" option), use as pairs to add proper postfix to output file
inputList =  open("Infiles/TTJets_SingleLeptFromT_TuneCP5_13TeV-madgraphMLM-pythia8.txt", "r") # tt + jets MC
thePostFix = "TTJets_SL"
#inputList =  open("Infiles/TTTT_TuneCP5_13TeV-amcatnlo-pythia8.txt", "r") # tttt MC
#thePostFix = "TTTT"
#inputList =  open("Infiles/TTTT_TuneCP5_PSweights_13TeV-amcatnlo-pythia8.txt", "r") # tttt MC PSWeights
#thePostFix = "TTTT_PSWeights"
#inputList =  open("Infiles/WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8.txt", "r") # W (to Lep + Nu) + jets
#thePostFix = "WJetsToLNu"

for line in inputList:
    #.replace('\n','') protects against new line characters at end of filenames, use just str(line) if problem appears
    files.append(filePrefix + str(line).replace('\n',''))

#Uncomment the two lines here to see what's being added to the list
#for file in files: 
#    print(file)

Below we'll create a single file, for speedier testing. Additionally, using root to open this file will let you browse its content. Use "new TBrowser" in root, once it has opened and attached the file.

In [10]:
#option to experiment with only the first file in the above list
onefile = [files[0]] 
print(onefile[0])

root://cms-xrd-global.cern.ch//store/mc/RunIIFall17NanoAOD/TTJets_SingleLeptFromT_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_94X_mc2017_realistic_v14-v1/00000/06CC0D9B-4244-E811-8B62-485B39897212.root


For running on Data, we would should do a few things: 

<ol>
    <li>Include a valid JSON file, which we'll define below. This rejects events from times when CMS's subdetectors were in unacceptable states.</li>
    <li>Switch our inputList to some valid data!</li>
    <li>Toggle the isData boolean inside the instance of our class (either manually above, or after creating our postprocessor below)</li>
</ol>

In [11]:
#For Monte Carlo:
theJSON = None

#For Data:
#theJSON = "Infiles/Cert_294927-306462_13TeV_EOY2017ReReco_Collisions17_JSON.txt"

Here we define the postprocessor with everything getting loaded, from files to JsonFile. "Named" options should have a default value, i.e. justcount's default is already False. Not all options are orthogonal.

 <ul>
  <li>outputDir: Defined above, this is where root files will be written. Can substitute "." for outputDir directly.</li>
  <li>jsonInput: json file in dictionary format {"RunNumberInt": [[lumilow,lumihigh],[lumi2low,lumi2high]], "RunNumber2Int":[[low,high]]}, using a string containing the relative path to the JSON's location</li>
  <li>files: list ["fileone.root","filetwo.root"] of inputs; even if just one file, must be a list! May point to a file to be accessed over XRD (make sure your proxy is valid!)</li>
<li>branchsel: if non-None, selection of branches to not even activate/load into memory, more efficient I/O and speed-wise</li>
<li>outputbranchsel: if non-None, selection of branches to keep/drop in output (see noOut)</li>
<li>noOut: If True, no output of skimmed data is written. If False, will write full data file fitting outputbranch selections and with postfix concattenated to name. Independant of histFile.</li>
<li>justCount: Just counts events fitting preselection criteria, analyze() criteria not taken into account</li>
<li>postfix: string added to inputfile name to indicate this module processed it!</li>
<li>histFileName: name of any output file for histograms created in your class, as above with runs and lumis</li>
<li>histDirName: name INSIDE the "histFileName.root" file's directory structure!</li>
<li>friend: Can be used to create "friend" trees to pair with original NanoAOD files.</li>
<li>provenance: not tested</li>
<li>haddFileName: Not properly tested (tied together with fwkJobReport)</li>
<li>fwkJobReport: only relevant with multiple files being added together? Not properly tested</li>
</ul> 

In [12]:
p=PostProcessor(outputDir,
                onefile,
                cut=preselection,
                branchsel=None,
                modules=[OurExampleModule()],
                friend=False,
                postfix=thePostFix, 
                jsonInput=theJSON,
                noOut=True,
                justcount=False,
                provenance=False,
                haddFileName=None,
                fwkJobReport=False,
                histFileName="OurModuleHist.root",
                histDirName="plotsDirectory", 
                outputbranchsel=None
               )

Here is an example of accessing 'externally' the class instance's variable isData

In [13]:
print("On instantiation, the isData boolean is...\nFlag isData = " + str(p.modules[0].isData))
#p.modules[0].isData = True #Access isData variable in the main class (loaded as first module in postprocessor)
print("\nSet to...\nFlag isData = " + str(p.modules[0].isData))

On instantiation, the isData boolean is...
Flag isData = False

Set to...
Flag isData = False


Before executing, let's set a limit on the number of events to run our full code over, as the default of -1 means all:

In [14]:
print("Initialized Event Limit: " + str(p.modules[0].EventLimit))
p.modules[0].EventLimit = 10
print("Set Event Limit: " + str(p.modules[0].EventLimit))

Initialized Event Limit: -1
Set Event Limit: 10


Now that everything is defined, we'll actually run the process and see its direct output:

In [15]:
p.run()

Pre-select 636713 entries out of 696370 


/afs/cern.ch/work/n/nmangane/scratch1/CMSSW_9_4_9/python/PhysicsTools/NanoAODTools/postprocessing/framework/treeReaderArrayTools.py:69: RuntimeWarning: creating executor for unknown type "ULong64_t*"
  ret = _vr.Get()[0]




Run:        1 	LuminosityBlock:       59 	Event:   167594
PV  X: -0.026 Y: 0.067 Z: 1.515 nDoF: 148.000 Chi^2: 0.814
|| Secondary Vertices	Number:   3 					    ||
||	   Pt	  Eta	  Phi	 Chi2	 nDoF	 Mass	 dLen		    ||
||	8.109	-0.206	-0.351	0.401	6.750	1.988	0.035		    ||
||	48.656	-0.392	-0.429	2.039	4.297	2.598	0.220		    ||
||	27.000	1.325	-1.658	1.301	6.094	2.936	2.328		    ||
|| Muons		Number:   2					    ||
||	   Pt	  Eta	  Phi	 IP3d	  dXY	   dZ	JetID	PFRelIso04  ||
||	40.590	-1.030	0.103	0.000	0.000	-0.000	    2	0.024	    ||
||	16.977	1.275	-1.651	0.086	0.011	0.165	    3	0.156	    ||
|| Electrons		Number:   1					    ||
||	   Pt	  Eta	  Phi	 IP3d	  dXY	   dZ	JetID	PFRelIso03  ||
||	4.987	-1.968	0.051	0.007	0.006	0.013	    7	0.601	    ||
|| Photons		Number:   0					    ||
|| AK4 Jets		Number:  10					    ||
||	   Pt	  Eta	  Phi	CSVv2	 CMVA	JetID	ChEmEF	ChHEF       ||
||	139.500	0.559	-3.032	0.069	-0.979	    6	0.000	0.625	    ||
||	88.062	-0.325	-0.408	0.991	0.997	    6	0.000	0.76